In [ ]:
# 1 패키지 설치
%pip install openai

---

# 2 OAI 연결
from openai import OpenAI

# 권한 인증 API

client = OpenAI(
        api_key=""
)

---

# 3 OpenAI API 호출(Completion 자동완성)

response = client.completions.create(
    model="",
    prompt="",
    max_tokens=16,
    stop=[],
    temperature=1.0,
    n=1,
)


---

# 4. OpenAI API 호출(ChatGPT) 

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    max_tokens=256,
    temperature=1.0,
    messages=[
        {"role":"user", "content":""}
    ]
)
print(response.choices[0].message.content)

---

# 5. Fewshot learning

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=1.0,
    max_tokens=200,
    messages=[
        {"role":"system", "content":"친절하고 자상한 고객상담원"},
        {"role":"assistant", "content":"항상 고객의 입장에서 생각하는 S센터입니다, 고객님 무엇을 도와드릴까요?"},
        {"role":"user", "content":"핸드폰이 작동 안되는데 AS 가능한가요?"},
        {"role":"assistant", "content":"불편을 끼쳐드려 대단히 죄송합니다, 고객님. AS 기간 확인을 위해 구매일자 확인이 필요합니다."},
        {"role":"user", "content":"구매한지 1주일 밖에 안됐어요"},
        {"role":"assistant", "content":"구매한지 얼마되지 않아 고장나서 정말 실망이 크셨겠네요, 오늘 AS 센터 방문하시면 무상 수리됩니다."},
        {"role":"user", "content":"택배 안되나요?"}
    ]
)

print(response.choices[0].message.content)


----

# 6. 캐릭터 생성

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=1.0,
    max_tokens=200,
    messages=[
        {"role":"system", "content":"항상 반발로 손님에게 말씀하시는 욕쟁이 식당 할머니"},
        {"role":"assistant", "content":"왜 왔냐?"},
        {"role":"user", "content":"할머니 배고파요. 백반 주세요"},
        {"role":"assistant", "content":"지금 시간이 몇신데 이제 밥을 먹냐"},
        {"role":"user", "content":"할머니 따뜻한 물도 한잔 주세요"},
        {"role":"assistant", "content":"날이 이렇게 더운데 따뜻한 물을 마시겠다고?"},
        {"role":"user", "content":"할머니 카드 결재 되나요?"}
    ]
  )

print(response.choices[0].message.content)

In [ ]:
# 1. 환경 설정

import os
from openai import OpenAI

# 환경 변수 설정!! Finetuning
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()

----

# 2. Fine-tuning 데이터 준비(.csv -> .jsonl)

!openai tools fine_tunes.prepare_data --file FT_InitialSound_Dataset.csv --quiet

---

# 3. Fine-tune 파일 업로드

training_file_name = "FT_InitialSound_Dataset_prepared.jsonl"

training_response = client.files.create(
    file=open(training_file_name, "rb"),
    purpose="fine-tune"
)

training_file_id = training_response.id
print(training_file_id)

---

# 4. FineTuning 작업 생성

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="",
    suffix="",
    hyperparameters={"n_epochs":4}
)

job_id = response.id

print(job_id)

---

# 5. 모델 학습 진행상황 확인

model_response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_status = model_response.status
print(fine_tuned_model_status)

---

# 6. 학습 끝난 모델 조회
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

print("Fine-tuned model ID:", fine_tuned_model_id)
# 사용하는 모델의 이름
# ex. ft:davinci-002:mijeongjeon:is-jmj-1009:87i94Oh1

---

# 7-1. Completion Base 모델 사용
completion = client.completions.create(
    model="text-davinci-003",
    temperature=0.1,
    max_tokens=10,
    stop=["\n"],
    prompt="ㄱㄱ ->"
)

completion.choices[0].text

--

# 7-2. Finetuned 모델 사용
completion = client.completions.create(
    model="",
    temperature=0.1,
    max_tokens=10,
    stop=["\n"],
    prompt="ㄱㄱ ->"
)

completion.choices[0].text

---

# 8-1. Chat Base 모델 테스트

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system", "content":"먹을 수 있는 음식 이름을 알려주면 칼로리양과 주요 영양소 탄수화물, 단백질, 지방 함량을 알려주고 음식 이모지를 붙여주는 친절한 도우미"},
        {"role":"user", "content":"떡볶이"}
    ]
)

print(response.choices[0].message.content)

---

# 8-2. FineTuned Chat 모델 테스트

response = client.chat.completions.create(
    model="",
    messages=[
        {"role":"system", "content":"먹을 수 있는 음식 이름을 알려주면 칼로리양과 주요 영양소 탄수화물, 단백질, 지방 함량을 알려주고 음식 이모지를 붙여주는 친절한 도우미"},
        {"role":"user", "content":"떡볶이"}
    ]
)

print(response.choices[0].message.content)